# CheckPointer demos

The notebooks below demostrates the convepts of basic chat, memory, tool use, retrieval, structured output, streaming, and multi-agent routing.

Each demo ends with a short **practice assignment**.


In [3]:
# Imports – common to all demos
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage, AIMessage

from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.prebuilt import ToolNode
from langchain_core import tools

# Groq LLM wrapper (compatible with LangChain 1.x)
def get_llm():
    return ChatGroq(model="llama-3.1-8b-instant", temperature=0.2)


## Demo-1 – Simple forgetful chatbot

Core concept: **single-turn LLM call**.

- **Scenario**: A customer asks for the current balance of a checking account.
- **Prompt**: Provide a concise answer without disclosing internal IDs.


In [4]:
def simple_balance_chat(user_input: str) -> str:
    llm = get_llm()
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a helpful banking assistant. Answer only the question asked. Do not ask for additional information."),
        ("human", "{question}")
    ])
    chain = prompt | llm
    return chain.invoke({"question": user_input}).content

# Example run
print(simple_balance_chat("What is my checking account balance?"))


I'm unable to access your account information. However, I can help you check your balance online or by calling our customer service number. Would you like me to provide you with that information?


In [6]:
print(simple_balance_chat("ok. we can start with that"))

It seems you're ready to begin. Is there anything I can assist you with today?


## Demo Setup – Define node and workflow


In [7]:
import operator
from typing import Annotated, Sequence, TypedDict

from langchain_core.messages import BaseMessage, HumanMessage, AIMessage

from langgraph.checkpoint.memory import MemorySaver
from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.graph import StateGraph, START, END

# Define the state for the graph (conversational memory)
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]

# Define the LLM
llm = get_llm()

# The single node in our graph: a banking assistant
def bank_assistant_node(state: AgentState):
    """
    This node processes user messages and generates a banking-related response.
    It leverages conversational history from the state.
    """
    messages = state['messages']
    # Prompt for a banking context
    prompt = "You are a helpful personal finance assistant at a bank. Respond to the user's query regarding their accounts, transactions, or general financial advice. Be concise and professional."
    
    # LangChain 1.x syntax for invoking the LLM
    response = llm.invoke([HumanMessage(content=prompt)] + messages)
    return {"messages": [response]}

# Build the graph
workflow = StateGraph(AgentState)
workflow.add_node("bank_assistant", bank_assistant_node)
workflow.add_edge(START, "bank_assistant")
workflow.add_edge("bank_assistant", END)

## Demo-1 – Using InMemory Saver

In [8]:
# Compile the graph with InMemorySaver
# This is perfect for single-session conversations or ephemeral testing
memory_checkpointer = MemorySaver()
app_in_memory = workflow.compile(checkpointer=memory_checkpointer)

# Example usage with a thread ID
config = {"configurable": {"thread_id": "user-session-1"}}

# Interaction 1
response1 = app_in_memory.invoke(
    {"messages": [HumanMessage(content="Hi, I want to check my account balance.")]},
    config=config
)
print(f"AI: {response1['messages'][-1].content}")

# Interaction 2 (memory persists within the 'user-session-1' thread)
response2 = app_in_memory.invoke(
    {"messages": [HumanMessage(content="Also, what was my last transaction?")]},
    config=config
)
print(f"AI: {response2['messages'][-1].content}")

AI: To check your account balance, you can:

1. Log in to your online banking account through our website or mobile app.
2. Call our 24/7 customer service number at 1-800-BANK-123 (1-800-226-5123).
3. Visit any of our branches in person.

If you need assistance with logging in or have any issues, please let me know and I'll be happy to help.
AI: I can assist you with that. Can you please provide me with your account number or the last 4 digits of your debit card so I can look up your account information? This will ensure I provide you with the most accurate and up-to-date information.

Alternatively, if you have already logged in to your online banking account, you can view your transaction history directly.


## Demo-2 – Persisting conversations in database


In [10]:
import sqlite3
from langgraph.checkpoint.sqlite import SqliteSaver
from langchain_core.messages import HumanMessage

# ... (previous code for AgentState, llm, bank_assistant_node, workflow, etc. remains the same) ...

# 1. Create a connection object first
db_path = "./banking_memory.db"
conn = sqlite3.connect(db_path, check_same_thread=False) # check_same_thread=False is important for multi-threading scenarios in a notebook

# 2. Instantiate the SqliteSaver with the connection object
sqlite_checkpointer = SqliteSaver(conn)

# 3. Compile the graph with the correct checkpointer instance
app_sqlite = workflow.compile(checkpointer=sqlite_checkpointer)

# Example usage with a thread ID
config_sql = {"configurable": {"thread_id": "user-session-sql-789"}}

# Interaction 1 (saved to the SQLite database file)
response_sql1 = app_sqlite.invoke(
    {"messages": [HumanMessage(content="I need to know my credit card limit.")]},
    config=config_sql
)
print(f"AI: {response_sql1['messages'][-1].content}")

# Interaction 2 (memory retrieved from the DB for the same thread)
response_sql2 = app_sqlite.invoke(
    {"messages": [HumanMessage(content="Is that a Visa or Mastercard?")]},
    config=config_sql
)
print(f"AI: {response_sql2['messages'][-1].content}")

AI: To retrieve your credit card limit, I'll need to verify your account information. Can you please provide me with your credit card number or the last 4 digits of the card, along with your date of birth? This will allow me to securely access your account details.
AI: To provide you with your credit card limit, I'll need to access your account information. However, I can try to assist you without needing your card number or date of birth. Can you please check your most recent credit card statement or online banking account to find your credit card limit? If you're still unable to locate it, I can try to look it up for you with your account name and address.
